## Streamlit 기반 사용자 가입 시스템 및 맞춤형 프롬프트 구성
- 목표: 이번 수업에서는 Streamlit을 활용하여 다음 기능을 구현한다:
    + 1) 간단한 사용자 회원가입 시스템을 구현
    + 2) 사용자의 데이터를 `JSON` 파일에 저장하고 불러오는 방법 학습
    + 3) LangChain 프롬프트 템플릿에 사용자 정보를 동적으로 반영하는 방법
    
- 링크: https://youtu.be/PnumBfdbO-k

---

### 학습 내용
1. JSON 파일을 활용한 사용자 데이터 저장
    + `data/user_profiles.json` 파일이 존재하지 않을 경우 새로 생성
    + `json.dump()`를 활용한 사용자 정보 저장
2. 사용자 정보 불러오기
    + `json.load()`를 사용해 기존 회원 정보를 로딩
    + 중복된 ID 체크 및 로그인 구현
3. Streamlit 폼 구성 및 상태관리
    + `st.from()`을 활용한 회원가입 입력 폼 구성
    + 입력값 검증 (비밀번호 잋리 여부, 필수 입력 확인 등)
    + `st.session_State`를 통한 세션 정보 관리
4. 사용자 정보 -> LangChain 프롬프트 템플릿 연계
    + 등록된 사용자 정보를 기반으로 프롬프트템플릿 작성

---

### 1. JSON 파일을 활용한 사용자 데이터 저장

#### 순서 1. 새로운 프로젝트(09-MiniProject)의 구조를 다음과 같이 설정한다.

+ `root` 경로
    - `09-MiniProject` : 새로운 프로젝트 경로
        * `pages` : 페이지 구성
            - `0_sign_in.py` : 로그인 페이지
            - `1_sign_up.py` : 회원가입 페이지
            - `3_main_chat.py` : 메인 채팅 페이지
        * `main.py` : 랜딩 페이지

![](https://codetutorbot.blob.core.windows.net/image/W8/mini01.png)

---

#### 순서 2. `1_sign_up.py`에서 `09-MiniProject/user_profiles.json` 경로에 파일(JSON)이 존재하지 않을 경우 새로 생성
##### 1) `1_sign_up.py`에 다음과 같이 코드를 작성한다.
![](https://codetutorbot.blob.core.windows.net/image/W8/mini02.png)

##### 2) `09-MiniProject` 경로에서 streamlit 앱을 실행한다.
+ cd 09-MiniProject
+ poetry run streamlit run main.py

#### 3) 앱 실행 => sign up 이동 => 프로젝트 경로에 `data/user_profiles.json`이 생성된 것을 확인한다.
![](https://codetutorbot.blob.core.windows.net/image/W8/mini03.png)

#### 4) `st.from`을 사용하여, `1_sign_up.py`에서 프론트 페이지를 구성하자.
- st.form은 Streamlit 앱에서 여러 입력 필드를 묶어서 하나의 제출 버튼으로 처리할 수 있도록 도와주는 기능.
- 입력 필드를 그룹화하고 하나의 submit 버튼으로 처리
- 폼 내의 모든 입력값은 st.form_submit_button()이 눌렸을 때만 반영됨
![](https://codetutorbot.blob.core.windows.net/image/W8/mini08.png)


#### 5) `load_user_profiles():`사용자 정보를 저장한 JSON 파일(user_profiles.json)이 존재할 경우, 저장된 데이터를 읽어와 딕셔너리 형태로 반환해보자.만약 파일이 존재하지 않으면, 빈 사용자 목록 {"users": {}}을 기본값으로 반환한다.
![](https://codetutorbot.blob.core.windows.net/image/W8/mini05.png)


#### 6) `save_user_profile(user_id, profile_data)`: 새로운 사용자 정보(`profile_data`)를 `user_id`를 key로 하여 JSON 파일에 저장한다. 기존 사용자 정보는 유지한 채, 새로운 사용자만 추가해서 파일에 다시 쓰는 방식이다.
![](https://codetutorbot.blob.core.windows.net/image/W8/mini06.png)



#### 7) 회원가입 폼에서 제출 버튼이 눌리면,
+ 1. 입력값이 모두 채워졌는지 확인
+ 2. 비밀번호가 일치하는지 확인
+ 3. ID가 이미 사용 중인지 확인
+ 4. 위 조건이 모두 만족되면 사용자 정보를 저장하고 다음 페이지(`main.py`)로 이동
![](https://codetutorbot.blob.core.windows.net/image/W8/mini07.png)

---


### 2. 메인 진입점을 `0_sign_in.py`로 설정하기
+ 다음 코드를 `main.py`에 작성합니다.

![](https://codetutorbot.blob.core.windows.net/image/W8/mini09.png)

---

### 3. 로그인 `0_sign_in.py`에 로그인 기능 구현하기
#### 1) `0_sign_in.py`에 프론트 화면을 구현하기 위해 다음 코드를 추가한다.

In [ ]:
# 0_sign_in.py
import streamlit as st
import json
from pathlib import Path

# 페이지 설정
st.set_page_config(
    page_title="Sign In",
    page_icon="🤖"
)

# 데이터 저장 경로 설정
DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)
USER_PROFILE_PATH = DATA_DIR / "user_profiles.json"

# 사용자 정보 파일을 읽어와 딕셔너리로 반환하는 함수
def load_user_profiles():
    if USER_PROFILE_PATH.exists():
        with open(USER_PROFILE_PATH, "r", encoding="utf-8") as f:
            return json.load(f)
    return {"users": {}}

def main():
    # 페이지 제목 출력
    st.title("🤖 Welcome!")

    # 안내 문구 출력
    st.markdown("""
    To communicate with BOT, please sign up and sign in first.
    """)

    # 로그인 폼 시작
    with st.form("login_form"):
        st.header("Login")  # 로그인 폼 제목

        # 사용자 입력 필드: ID와 비밀번호
        username = st.text_input("ID")
        password = st.text_input("Password", type="password")

        # 버튼을 두 열로 나누어 배치
        col1, col2 = st.columns(2)
        with col1:
            # 로그인 버튼
            login_submitted = st.form_submit_button("Login")
        with col2:
            # 회원가입 버튼 클릭 시 회원가입 페이지로 이동
            if st.form_submit_button("Sign Up"):
                st.switch_page("pages/1_sign_up.py")

if __name__ == "__main__":
    main()

#### 2) 사용자 인증 함수(`authenticate_user(username, password)`)-ID와 비밀번호가 맞는지 확인, 정의하기

+ 다음 코드를 추가해보자.

![](https://codetutorbot.blob.core.windows.net/image/W8/mini11.png)

#### 3) 로그인 기능 추가하기

+ 1. Streamlit에서 로그인 폼을 만들고,

+ 2. 사용자 입력값(ID, 비밀번호)을 받고,

+ 3. 로그인 버튼이 눌렸을 때 입력값 검증 및 인증을 진행하며,

+ 4. 성공 시 세션 상태를 업데이트하고 채팅 페이지로 이동합니다.

+ 5. 실패 시 에러 메시지를 출력합니다.

![](https://codetutorbot.blob.core.windows.net/image/W8/mini12.png)

---

### 4. 로그인 `3_main_chat.py`에 로그인한 사용자의 정보를 불러와 인사 메시지 출력하기

- #### `3_main_chat.py`에 다음 코드를 추가한다.

In [ ]:
# 3_main_chatp.py
import streamlit as st
import json
from pathlib import Path

# 페이지 기본 설정 (탭 제목 및 아이콘 지정)
st.set_page_config(
    page_title="Main Chat",
    page_icon="🤖"
)

# 사용자 정보가 저장된 파일 경로 설정
DATA_DIR = Path("data")
USER_PROFILE_PATH = DATA_DIR / "user_profiles.json"

# 사용자 ID로부터 사용자 정보를 불러오는 함수
def load_user_profile(user_id):
    with open(USER_PROFILE_PATH, "r", encoding="utf-8") as f:
        profiles = json.load(f)  # JSON 파일을 딕셔너리로 읽어옴
        return profiles["users"].get(user_id)  # 해당 user_id의 정보 반환

def main():
    # 세션에 로그인 정보가 없으면 메인 페이지(로그인)로 이동
    if "user_id" not in st.session_state or not st.session_state.user_id:
        st.switch_page("main.py")
        return

    # 사용자 ID로 프로필 불러오기
    user_profile = load_user_profile(st.session_state.user_id)

    # 사용자에게 환영 메시지 출력
    st.write(f"Welcome, {user_profile['id']}!")

# 프로그램의 진입점: main() 함수 실행
if __name__ == "__main__":
    main()
